# Pandas Introduction

## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [7]:
def answer_one():
    """
    Return the name of the country which has won the most
    gold medals during the summer games.
    """
    df_copy = df.copy()
    df_copy['Country'] = df_copy.index
    df_copy.reset_index(inplace=True,drop=True)
    
    return df_copy.loc[df_copy['Gold'].idxmax()]['Country']
print('Country with most gold medals during summer games: {}.'.format(answer_one()))

Country with most gold medals during summer games: United States.


### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [10]:
def answer_two():
    """
    Country with the biggest difference between their
    summer and winter gold medal counts
    """
    df_copy = df.copy()
    df_copy['Country'] = df_copy.index
    df_copy.reset_index(inplace=True,drop=True)
    df_copy['Diff'] = df_copy['Gold'] - df_copy['Gold.1']
    
    return df_copy.loc[df_copy['Diff'].idxmax()]['Country']
print('Country with biggest difference between summer and winter gold medal counts: {}.'.format(answer_two()))

Country with biggest difference between summer and winter gold medal counts: United States.


### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [13]:
def answer_three():
    """
    Country with biggest ratio.
    """
    df_copy = df.copy()
    df_copy['Country'] = df_copy.index
    df_copy.reset_index(inplace=True,drop=True)
    
    mask = (df_copy['Gold']>=1) & (df_copy['Gold.1']>=1)
    df_copy = df_copy[mask]
    
    df_copy['Diff'] = (df_copy['Gold'] - df_copy['Gold.1']) / df_copy['Gold.2']

    
    return df_copy.loc[df_copy['Diff'].idxmax()]['Country']
print('Country with largest summer to winter gold medal counts: {}.'.format(answer_three()))

Country with largest summer to winter gold medal counts: Bulgaria.


### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [15]:
def answer_four():
    points = df['Gold.2']*3+df['Silver.2']*3+df['Bronze.2']
    points.name='Points'
    return points
answer_four()[0:5]

Afghanistan      2
Algeria         29
Argentina      154
Armenia         18
Australasia     26
Name: Points, dtype: int64

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [17]:
census_df = pd.read_csv('./data/census.csv')
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [19]:
def answer_five():
    """
    Return state with most counties.
    """
    df = census_df[census_df['SUMLEV']==50]
    return df[['STNAME','COUNTY']].groupby('STNAME').count().idxmax().values[0]

print('The state with the most counties is:')
answer_five()

The state with the most counties is:


'Texas'

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [22]:
def answer_six():
    """
    Using only the three most populated counties, returns the state with
    the highest population count.
    """
    # retrieve only county data
    df = census_df[census_df['SUMLEV']==50][['STNAME','CENSUS2010POP']].set_index('STNAME',drop=True)
    
    df_filtered = pd.DataFrame(columns=['STNAME','POPULATION'])
    # find three most populated counties per state
    for state in df.index.unique():
        if type(df.loc[state,'CENSUS2010POP']) is np.int64:
            state_pop = df.loc[state,'CENSUS2010POP']
        elif len(df.loc[state,'CENSUS2010POP'])<3:
            state_pop = sum(df.loc[state,'CENSUS2010POP'])
        else:
            state_pop = df.loc[state,'CENSUS2010POP'].sort_values(ascending=False)[0:3].sum()
        
        df_filtered = df_filtered.append({'STNAME':state,'POPULATION':state_pop},ignore_index=True)
    
    df_filtered = df_filtered.sort_values(by='POPULATION',ascending=False).head(3)
    
    # return index
    return list(df_filtered['STNAME'].values)

print('The three most populated states are: {0}, {1}, and {2}'.format(*answer_six()))

The three most populated states are: California, Texas, and Illinois


### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [24]:
def answer_seven():
    """
    Return the county with the largest absolute change in population within the
    period 2010-2015.
    """
    df = census_df[census_df['SUMLEV']==50]
    df = df.set_index('CTYNAME',drop=True)
    
    df['Max'] = np.max(df[['POPESTIMATE2010','POPESTIMATE2011',
                    'POPESTIMATE2012','POPESTIMATE2013',
                    'POPESTIMATE2014','POPESTIMATE2015']],axis=1)
    
    df['Min'] = np.min(df[['POPESTIMATE2010','POPESTIMATE2011',
                    'POPESTIMATE2012','POPESTIMATE2013',
                    'POPESTIMATE2014','POPESTIMATE2015']],axis=1)
    
    df['Change'] = df['Max']-df['Min']
    
    max_change = df['Change'].idxmax()
    
    return max_change

print('The county with the largest absolute change in population within the period 2010-2015 is:')
answer_seven()

The county with the largest absolute change in population within the period 2010-2015 is:


'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [26]:
def answer_eight():
    """
    Return the counties with meet the following requierements:
     - belong to region 1 or 2
     - name starts with 'W'
     - POPESTIMATES > POPESTIMATE2014
    """
    query = (census_df['REGION'].isin([1,2])) & (census_df['CTYNAME'].map(lambda x: x.startswith('Washington'))) & (census_df['POPESTIMATE2015'] > census_df['POPESTIMATE2014'])
    
    df = census_df[query]
    
    return df[['STNAME', 'CTYNAME']].sort_index()

answer_eight()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County


# More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [16]:
import pandas as pd
import numpy as np

ENERGY_FILE = './data/Energy+Indicators.xls'
GDP_FILE = './data/world_bank.csv'
SCIAMGO_FILE = './data/scimagojr-3.xlsx'

In [17]:
def get_GDP():
    # load gdp data
    GDP = pd.read_csv(GDP_FILE,
                         skiprows=4)
    
    # rename column
    GDP = GDP.rename(columns={'Country Name':'Country'})
    
    # rename countries
    GDP['Country'] = GDP['Country'].str.replace('Korea, Rep.', 'South Korea')
    GDP['Country'] = GDP['Country'].str.replace('Iran, Islamic Rep.', 'Iran')
    GDP['Country'] = GDP['Country'].str.replace('Hong Kong SAR, China', 'Hong Kong')

    # filter dataframe before merge:
    # last 10 years of GDP data
    GDP  = GDP[['Country','2006','2007','2008','2009',
               '2010','2011','2012','2013','2014','2015']]
    
    return GDP

def get_energy():
    # load energy incidators files
    energy = pd.read_excel(ENERGY_FILE,
                       skiprows=17,
                       skipfooter=38,
                       usecols=[2,3,4,5,6],
                       index_col=0,
                       na_values='...',
                       names = ['Energy Supply', 'Energy Supply per Capita', '% Renewable'])
    
    # reset index
    energy.index.name = 'Country'
    energy = energy.reset_index()
    
    # convert supply
    energy['Energy Supply'] = energy['Energy Supply'] * 10**6
    
    # remove digits in index
    energy['Country'] = energy['Country'].str.replace("\d+", "")
    
    # remove (...) from index
    energy['Country'] = energy['Country'].str.split(' \(').str[0].str.strip()
    
    # rename countries
    energy['Country'] = energy['Country'].str.replace('Republic of Korea','South Korea')
    energy['Country'] = energy['Country'].str.replace('United States of America','United States')
    energy['Country'] = energy['Country'].str.replace('United Kingdom of Great Britain and Northern Ireland','United Kingdom')
    energy['Country'] = energy['Country'].str.replace('China, Hong Kong Special Administrative Region','Hong Kong')
    
    return energy

def get_ScimEn():
    # load scimago file
    ScimEn = pd.read_excel(SCIAMGO_FILE)
    
    return ScimEn

In [18]:
def answer_one():
    
    # load data
    energy, GDP, ScimEn = get_energy(), get_GDP(), get_ScimEn()
        
    # Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names).
    df_merge = pd.merge(left=GDP,right=energy,
                        how='inner',left_on='Country',
                        right_on='Country')
    df_merge = pd.merge(left=df_merge,right=ScimEn,
                        how='inner',left_on='Country',
                        right_on='Country')
    
    # filter to only maintain the country with a Rank <= 15
    df_merge = df_merge[df_merge['Rank']<=15]
    
    df_merge = df_merge.set_index('Country',drop=True)
    
    return df_merge

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [19]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [20]:
def answer_two():
    # load data
    energy, GDP, ScimEn = get_energy(), get_GDP(), get_ScimEn()
    
    tdf_in = pd.merge(left=GDP, right=energy, on='Country',how='inner')
    tdf_in = pd.merge(left=tdf_in, right=ScimEn, on='Country',how='inner')
    
    tdf_out = pd.merge(left=GDP, right=energy, on='Country',how='outer')
    tdf_out = pd.merge(left=tdf_out, right=ScimEn, on='Country',how='outer')

    return len(tdf_out) - len(tdf_in)

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [21]:
def answer_three():
    Top15 = answer_one()
    
    years = [str(x) for x in range(2006,2016)]
    results = Top15[years].mean(axis=1).sort_values(ascending=False)
    results.name = 'avgGDP'
    return results

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [22]:
def answer_four():
    # load data
    Top15 = answer_one()
    
    # find 6th largest GDP
    sixth_gdp = answer_three().sort_values(ascending=False).iloc[5]
    sixth_country = answer_three()[answer_three()==sixth_gdp].index[0]
    
    return Top15.loc[sixth_country,'2015'] - Top15.loc[sixth_country,'2006']

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [23]:
def answer_five():
    Top15 = answer_one()
    return Top15['Energy Supply per Capita'].mean()

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [24]:
def answer_six():
    Top15 = answer_one()
    return (Top15['% Renewable'].idxmax(),Top15['% Renewable'].max())

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [25]:
def answer_seven():
    Top15 = answer_one()
    Top15['Ratio'] = Top15['Self-citations'] / Top15['Citations']
    return (Top15['Ratio'].idxmax(),Top15['Ratio'].max())

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [26]:
def answer_eight():
    Top15 = answer_one()
    Top15['Pop'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    return Top15['Pop'].nlargest(3).iloc[[-1]].index[0]

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [27]:
def answer_nine():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    
    corr = Top15[['Citable docs per Capita','Energy Supply per Capita']].corr()
    corr = corr.loc['Energy Supply per Capita', 'Citable docs per Capita']
    return corr

In [28]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [30]:
def answer_ten():
    Top15 = answer_one()
    Top15['Median'] = Top15['% Renewable']>=Top15['% Renewable'].median()
    Top15['Median'] = Top15['Median'].astype(float)
    HighRenew = Top15.loc[:,['Rank','Median']].sort_values('Rank',ascending=True)
    HighRenew = HighRenew['Median']
    HighRenew.name = 'HighRenew'
    return HighRenew

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [31]:
ContinentDict  = {'China':'Asia', 
              'United States':'North America', 
              'Japan':'Asia', 
              'United Kingdom':'Europe', 
              'Russian Federation':'Europe', 
              'Canada':'North America', 
              'Germany':'Europe', 
              'India':'Asia',
              'France':'Europe', 
              'South Korea':'Asia', 
              'Italy':'Europe', 
              'Spain':'Europe', 
              'Iran':'Asia',
              'Australia':'Australia', 
              'Brazil':'South America'}

In [32]:
def answer_eleven():
    Top15 = answer_one()
    Top15['Continent'] = Top15.index
    
    Top15['Continent'] = Top15['Continent'].apply(lambda x: ContinentDict[x])
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15 = Top15.groupby('Continent')['PopEst'].agg({'size': np.size, 'sum': np.sum, 'mean': np.average, 'std': np.std})
    
    return Top15

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [33]:
def answer_twelve():
    Top15 = answer_one()
    Top15['Continent'] = Top15.index
    Top15['Continent'] = Top15['Continent'].apply(lambda x: ContinentDict[x])
    Top15['group'] = pd.cut(Top15['% Renewable'], 5)
    Top15 = Top15.groupby(['Continent','group']).size()
    
    return Top15

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [34]:
def answer_thirteen():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['PopEst'] = Top15['PopEst'].apply(lambda x:'{:,}'.format(x))
    return Top15['PopEst']

# Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [35]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [36]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # create empty DataFrame
    df_towns = pd.DataFrame(columns=["State", "RegionName"])
    
    # file name
    file_name = 'university_towns.txt'
    
    # data extraction
    with open(file_name,'r') as f:
        state = ''
        region = ''
        for row in f:
            # check if contains state or region
            if '[e' in row:
                # save state
                state = row.split('[ed')[0].strip()
                continue
            region = row.strip()
            if '(' in region:
                region = row.split('(')[0].strip()
            # populate DataFrame
            df_towns = df_towns.append({'State':state,'RegionName':region},ignore_index=True,)        
    
    return df_towns

In [37]:
def get_gdp_data():
    """ Returns a clean dataframe containing the GDP data per quarter starting in 2000q1"""
    
    # file name
    file_name = 'gdplev.xls'
    
    # load quarterly data and clean data
    df_gdp = pd.read_excel(file_name,skiprows=5,
                           index_col=0,parse_cols=[4,5,6]).dropna()
    
    index_start = df_gdp.index.get_loc('2000q1')
    
    # filter data to post 2000
    df_gdp = df_gdp.ix[index_start:]
    
    return df_gdp

In [38]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    # recession dollars
    dollars = 'GDP in billions of chained 2009 dollars'
    
    # load gdp data
    df_gdp = get_gdp_data()[dollars]
    
    # find recession start
    for i in range(1,len(df_gdp)-1):
        if (df_gdp.iloc[i]<df_gdp.iloc[i-1]) and (df_gdp.iloc[i+1]<df_gdp.iloc[i]):
            return df_gdp.index[i]
    
    return None

In [39]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    # recession dollars
    dollars = 'GDP in billions of chained 2009 dollars'
    
    # load gdp data
    df_gdp = get_gdp_data()[dollars]
    
    # keep data following recession start
    df_gdp = df_gdp.ix[get_recession_start():]
    
    # find recession end
    for i in range(2,len(df_gdp)-1):
        if (df_gdp.iloc[i]>df_gdp.iloc[i-1]) and (df_gdp.iloc[i-1]>df_gdp.iloc[i-2]):
            return df_gdp.index[i]
    
    return None

In [40]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    # recession dollars
    dollars = 'GDP in billions of chained 2009 dollars'
    
    # load gdp data
    df_gdp = get_gdp_data()[dollars]
    
    # recession starts
    start = get_recession_start()
    end = get_recession_end()
    
    # recession data
    df_gdp = df_gdp.ix[start:end]
    
    return df_gdp.idxmin()

In [41]:
def get_housing_data():
    """Return dataframe containing the housing data from Zillow."""
    
    # file name
    file_name = 'City_Zhvi_AllHomes.csv'
    
    # load housing data and clean data
    df_housing = (pd.read_csv(file_name)
                              .replace(to_replace='NaN', value=np.NaN)
                              .convert_objects(convert_numeric=True))
    
    # remove uneccessary features
    df_housing = df_housing.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'],axis=1)
    
    # convert state ID to state name
    df_housing = df_housing.replace(states)
    
    # set multi-index
    df_housing = df_housing.set_index(['State','RegionName'])
        
    # filter columns to only contain data starting from 2000
    start_col = list(df_housing.columns.values).index('2000-01')
    df_housing = df_housing.drop(df_housing.columns[:start_col], axis=1)
    
    return df_housing

In [42]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    # import data
    df_housing = get_housing_data().T
    
    # convert index to time-stamp
    df_housing.index = pd.to_datetime(df_housing.index)
    
    # resample index by quarter and aggregate using mean
    df_housing = df_housing.resample('Q').mean()
    
    # convert index to proper values
    df_housing.index = df_housing.index.map(lambda x: str(x.year)+'q'+str((x.month-1)//3+1))
    
    # transpose data frame
    df_housing = df_housing.T
    
    return df_housing

In [43]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # load data
    df_housing = convert_housing_data_to_quarters()
    
    # find start and bottom of recession
    start_idx = df_housing.columns.get_loc(get_recession_start())
    bottom_idx = df_housing.columns.get_loc(get_recession_bottom())
    
    #(price_ratio=quarter_before_recession/recession_bottom
    df_housing['Ratio'] = df_housing[df_housing.columns[start_idx - 1]].div(df_housing[df_housing.columns[bottom_idx]])
    
    # filter data to relevant feature
    df_housing = df_housing[['Ratio']]
    
    # load univercities cities
    df_univ = get_list_of_university_towns().set_index(['State','RegionName'])
    
    # Univercity cities prices
    df_univ_prices = pd.merge(left=df_housing,right=df_univ,how='inner',
                            left_index=True,right_index=True)
    
    # generate non-university cities
    non_univ = np.setdiff1d(df_housing.index,df_univ.index)
    
    # filter data to non-university cities
    df_non_univ_prices = df_housing.loc[non_univ]
    
    # drop Na
    df_univ_prices = df_univ_prices.dropna(axis=0)
    df_non_univ_prices = df_non_univ_prices.dropna(axis=0)
    
    # T-test threshold
    alpha = 0.01
    
    # perform T-test
    s, p = ttest_ind(df_univ_prices['Ratio'], df_non_univ_prices['Ratio'])
    
    # assign output variables
    different = p<alpha
    better = "university town" if s<0 else "non-university town"
    
    return (different, p, better)